In [1]:
# import os    # çalısma yolunu değiştirdim
# os.getcwd()


In [2]:
import os 
import cv2
import numpy as np
import pandas as pd


path="C:\\aaa\\trainn\\"


folderNames=os.listdir(path)
XSample,ySample=[],[]
for i,folderName in enumerate(folderNames):
    for j,file in enumerate([f for f in os.listdir(path+folderName) if os.path.isfile(os.path.join(path+folderName, f))]):
        img=cv2.resize(cv2.imread(path+folderName+"\\"+file),(128,128),interpolation=cv2.INTER_LINEAR)/255.
        XSample.append(np.asarray(img))
        ySample.append(i)
XSample=np.array(XSample)
ySample=np.array(ySample)
print(XSample.shape)
print(ySample.shape)

(1800, 128, 128, 3)
(1800,)


In [3]:
from sklearn.model_selection import train_test_split
# validationu  xtrainden bölüyoruz
XTrain,XTest,yTrain,yTest = train_test_split(XSample,ySample,test_size=0.10,random_state=0)
XTrain,XVal,yTrain,yVal=train_test_split(XTrain,yTrain,test_size=0.111,random_state=0)
print(XTrain.shape)
print(XTest.shape)
print(XVal.shape)

(1440, 128, 128, 3)
(180, 128, 128, 3)
(180, 128, 128, 3)


In [6]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
model=models.Sequential()

model.add(layers.Conv2D(32,(3,3), activation='relu',input_shape=(128,128,3)))# !128 64  32 16 lük iyi sonuc 98 96
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(16,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

# model.add(layers.Conv2D(32,(3,3), activation='relu'))# !64 32 (98 93)
# model.add(layers.MaxPooling2D((2,2)))

# model.add(layers.Conv2D(16,(3,3), activation='relu'))
# model.add(layers.MaxPooling2D((2,2)))

# model.add(layers.Conv2D(16,(3,3), activation='relu'))
# model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation='sigmoid')) 

model.summary()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])# multi class oldugu için binary entropy aldık


# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

history= model.fit(XTrain,yTrain, epochs=20,batch_size=10,
                   validation_data=(XVal, yVal))

# history=model.fit(XTrain,yTrain,batch_size=100,validation_data=(XVal,yVal),epochs=20) bach fazla okuyor 100 olunca eksik tamamlanıyor

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 61, 61, 16)        4624      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 30, 30, 16)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 14400)             0         
                                                                 
 dropout_2 (Dropout)         (None, 14400)            

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["acc"])
plt.plot(history.history["val_acc"])

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Accuracy/Loss")
plt.ylabel("Accuracy/Loss")
plt.xlabel("epoch")
plt.legend(["Train Acc",    "VAL Acc", "Train Loss",   "VAL Loss"],loc="upper left")
plt.show()


In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# yTestPred=np.argmax(yTestPred, axis=1) argmax kukllanmıyoruz çünkü amacı multi olan durumda en yüksek değerli sınıfı alır.
yTestPred=np.asarray(np.round(model.predict(XTest),0),dtype=int)  # çok önemli 
print(yTestPred.reshape(yTestPred.shape[0]))  #  cok önemli

yTest=np.asarray(yTest)
print(classification_report(yTest,yTestPred))
cm=confusion_matrix(yTest,yTestPred, normalize="true").round(2)

disp=ConfusionMatrixDisplay(confusion_matrix=cm)
disp=disp.plot(cmap=plt.cm.GnBu, values_format='g')
plt.title(f"Train Confussion Matrix", style="italic")
plt.show()

In [ ]:
# Çita ve sırtlan, Afrika savanlarının iki önemli yırtıcı hayvanıdır. Bu iki tür, farklı avlama noktası ve çevresindekilerle bilenler ve yapay zeka teknolojileri, özellikle makine bilgisayarlarını, bu türlerin etrafındakileri ve çevrelerini hakkında daha fazla bilgi içerenimize yardımcı olabilir.

# Cheetah, dünyanın en hızlı kara hayvanıdır. Yüksek Hız sayesinde avlarını yakalamak için sık sık koşular ve bu nedenle savanların ana yırtıcılarıdır. Cheetah'ların avlanma stratejisi, hızlarını kullanarak yakın mesafeden yakalamak için avlarını sıkıştırmak. Bununla birlikte, bu strateji yüksek bir enerji tüketimi gerektirir ve cheetah'lar uzun süre sürdürürken takip etmeleri zor olan avlarını kaybedebilirler.

# Hyena, daha büyük bir av sahasına sahip olan daha sosyal bir yırtıcıdır. Avlarını avlayabilmek için birçok farklı strateji kullanırlar ve genellikle diğer yırtıcılarla çekişirler. Sırtlanlar, özellikle aslanlarla olan rekabetleriyle ünlüdürler ve bazen aslanların avını çalarlar. Sırtlanların sosyal çevreleri, av sahaları ve çevreleri üzerinde önemli etkilere sahiptir.

# Yapay zeka teknolojileri, bu türlerin donanımları ve donanımları hakkında daha fazla bilgi birikimimize yardımcı olabilir. Makine çalışmasını, birçok veri kaynağından toplanan verileri analiz ederek, bu türlerin etrafındaki ve çevrelerindeki noktaları hakkında bilgi birikimimize yardımcı olabilir. Örneğin, cheetah'ların avlanma stratejilerini ve sırtlanların avlarını avlamak için hangi uzak mesafe lavabolarını yönetiriz.

# Ayrıca, yapay zeka teknolojileri, bu türlerin bölümleri ve yaşam alanları hakkında da bilgi verebilir. Yapay zeka, eşleştirme maaşı izleyerek, bu türlerin yaşam tarzını nasıl sürdüreceğini ve bu rehberleri üzerinde barındırmaya çalışır.

# Sonuç olarak, yapay zeka teknolojileri, çita ve sırtlan gibi yırtıcı hayvanlar hakkında daha fazla bilgi içeriğimize yardımcı olabilir. Bu teknolojiler, bu türlerin etrafındaki ve donanımları hakkında daha iyi bir anlayış sağlayarak, bu türlerin korunması ve sürdürülebilir bir gele

In [ ]:
# https://www.kaggle.com/code/davegn/classification-of-cheetah-vs-hyena#5.-Modelling    farklı kişinin farklı bir modellemesi

# 2 ci farklı trainleme

In [ ]:
# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import os
# import cv2
# import matplotlib.pyplot as plt
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.preprocessing import image
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
# train = ImageDataGenerator(rescale=1/255)
# val = ImageDataGenerator(rescale=1/255)

# 2 farklı tür modellemesi

In [ ]:
# train_dataset = train.flow_from_directory("C:\\aaa\\trainn\\", target_size=(200, 200), class_mode = 'binary')
# val_dataset = val.flow_from_directory("C:\\aaa\\valitation\\", target_size=(200, 200), class_mode = 'binary')

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import datasets, layers, models
# model = models.Sequential()

# model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(200,200,3)))
# model.add(layers.MaxPool2D(2,2))

# model.add(layers.Conv2D(64,(3,3),activation='relu'))
# model.add(layers.MaxPool2D(2,2))

# model.add(layers.Conv2D(128,(3,3),activation='relu'))
# model.add(layers.MaxPool2D(2,2))

# model.add(layers.Conv2D(128,(3,3),activation='relu'))
# model.add(layers.MaxPool2D(2,2))

# model.add(layers.Flatten())

# model.add(layers.Dense(512,activation='relu'))

# model.add(layers.Dense(1,activation='sigmoid'))


In [ ]:
# model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

# train aşaması

In [ ]:
# model.fit_generator(train_dataset, steps_per_epoch = 5, epochs = 3, validation_data = val_dataset)

In [ ]:
#  import tensorflow as tf
# from sklearn.metrics import classification_report,ConfusionMatrixDisplay,confusion_matrix
# import matplotlib.pyplot as plt
# import numpy as np

# yTestPred=model.predict(train_dataset)
# yTestPred=np.argmax(val_dataset,axis=1)# önemli kod
# yTest=np.asarray(val_dataset) # buda önemli kod
# print(classification_report(yTest,  yTestPred))
# cm=confusion_matrix(yTest,yTestPred,normalize="true").round(2)
# # cmn= cm.astype("float") / cm.sum(axis=1)[:,np.newaxis]
# # cmn=cmn.round(2)
# disp=ConfusionMatrixDisplay(confusion_matrix=cm)
# disp=disp.plot(cmap=plt.cm.GnBu,values_format="g")
# plt.title(f"Train Confusion_matrix",    style="italic")
# # plt.savefig(f"mode12_CM_8901_{self.header}_Data_son.jpg")
# plt.show()

In [ ]:
# def predict(filename):
#     img1 = image.load_img(filename, target_size=(200, 200))
    
#     plt.imshow(img1)
#     Y = image.img_to_array(img1)
#     X = np.expand_dims(Y, axis=0)
#     val = model.predict(X)
#     print(val)
#     if val == 1:
#         plt.xlabel("A Hyna")
#     elif val == 0:
#         plt.xlabel("A Cheetah")

In [ ]:
# predict('C:\\aaa\\trainn\\cheetah\\cheetah_000_resized.jpg')

In [ ]:
# predict('C:\\aaa\\trainn\\hyena\\hyena_000_resized.jpg')   

In [ ]:
# from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix
# import matplotlib.pyplot as plt
# import numpy as np

# # yTestPred=np.argmax(yTestPred, axis=1) argmax kukllanmıyoruz çünkü amacı multi olan durumda en yüksek değerli sınıfı alır.
# yTestPred=np.asarray(np.round(model.predict(XTest),0),dtype=int)  # çok önemli 
# print(yTestPred.reshape(yTestPred.shape[0]))  #  cok önemli

# yTest=np.asarray(yTest)
# print(classification_report(yTest,yTestPred))
# cm=confusion_matrix(yTest,yTestPred, normalize="true").round(2)

# disp=ConfusionMatrixDisplay(confusion_matrix=cm)
# disp=disp.plot(cmap=plt.cm.GnBu, values_format='g')
# plt.title(f"Train Confussion Matrix", style="italic")
# plt.show()

In [ ]:
# farklı bir veri ile calısma 